In [68]:
from collections import namedtuple
import time
import sys

import numpy as np
import numpy.linalg as la

In [69]:
class Route:
    def __init__ (self, origin=None, destination=None):
        self.origin = origin # write appropriate value
        self.destination = destination

    def __repr__(self):
        return f"{self.origin}-{self.destination}"
        

class Edge:
    def __init__ (self, origin=None, destination=None, weight=None):
        self.origin = origin
        self.destionation = destination
        self.weight = weight # total routes from origin to destination

    def __repr__(self):
        return "edge: {0} {1} {2}".format(self.origin, self.destination, self.weight)
        
    ## write rest of code that you need for this class

class Airport:
    def __init__ (self, iden=None, name=None, pageindex=None):
        self.code = iden
        self.name = name
        self.routes = []
        self.routeHash = dict()
        #self.outweight =    # write appropriate value
        self.pageIndex = pageindex

    def __repr__(self):
        return f"{self.code}\t{self.pageIndex}\t{self.name}"
    
    def get_id(self):
        return self.name

routeList = [] # list of Routes
routeHash = dict() # hash of routes to ease the match
edgeList = [] # list of Edge
edgeHash = dict() # hash of edge to ease the match
airportList = [] # list of Airport
airportHash = dict() # hash key IATA code -> Airport

def readAirports(fd):
    print("Reading Airport file from {0}".format(fd))
    airportsTxt = open(fd, "r");
    cont = 0
    for line in airportsTxt.readlines():
        a = Airport()
        try:
            temp = line.split(',')
            if len(temp[4]) != 5 :
                raise Exception('not an IATA code')
            else:
                a.name=temp[1][1:-1] + ", " + temp[3][1:-1]
                a.code=temp[4][1:-1]
                a.pageIndex=temp[0]
                airportList.append(a)
                airportHash[a.code] = cont #this is the output of that function
                cont += 1
        except Exception as inst:
            pass
    airportsTxt.close()
    print(f"There were {cont} Airports with IATA code")


def readRoutes(fd):
    print("Reading Routes file from {fd}")
    routesTxt = open(fd, "r");
    cont = 0
    for line in routesTxt.readlines():
        a = Route()
        try:
            temp = line.split(',')
            if len(temp[2]) != 3 :
                raise Exception('not an IATA code')
            a.origin=temp[2]
            a.destination=temp[4]
            
        except Exception as inst:
            pass
        else:
            cont += 1
            routeList.append(a)
            #routeHash[a.origin] = a
    
    routesTxt.close()
    print(f"There were {cont} Routes with IATA code")
    

def createEdge() :
    # create a list of unique airports
    list_unique_airports = []
    for obj in routeList:
        list_unique_airports.append((obj.origin, obj.destination))
    # get rid of duplicates
    list_unique_airports = list(dict.fromkeys(list_unique_airports))
    
    # go through list of airport and find their weights
    for obj in list_unique_airports:
        edge = Edge()
  
        listupdate = [x if x.origin == obj[0] and x.destination == obj[1] else None for x in routeList]
        listupdate = list(dict.fromkeys(listupdate))
        const = len(listupdate) - listupdate.count(None)
        
        edge.origin = obj[0]
        edge.destination = obj[1]
        edge.weight = const
        edgeList.append(edge)
        #edgeHash[edge.origin] = edge
    #take out the route that start or arrive to an airport that is not listed in the file airports
    
    print(f"The EdgeList was succesfully created in variable edgeList")

In [ ]:
def computePageRanks(routeMatrix, L=0.85):
    n = routeMatrix.shape[0]
    M = L * routeMatrix + (1-L)/n * np.ones([n, n])
    r = 100 * np.ones(n)/n # the all 1/n vector * 100
    last = r
    r = M @ r
    iterations = 0 # initialize iterations counter
    while la.norm(last - r) > 0.01:
        last = r
        r = M @ r
        iterations += 1
    return r, iterations


#def outputPageRanks():
    # write your code

In [ ]:
def main(argv=None):

    readAirports("airports.txt")
    readRoutes("routes.txt") #just extracted the connection without adding the weights
    createEdge()
    time1 = time.time()
    pr_matrix, iterations = computePageRanks()
    time2 = time.time()
    #outputPageRanks()
    #print("#Iterations:", iterations)
    #print("Time of compute PageRanks():", time2-time1)

In [70]:
# test
readAirports("airports.txt")
readRoutes("routes.txt") #just extracted the connection without adding the weights
createEdge()

Reading Airport file from airports.txt
There were 5740 Airports with IATA code
Reading Routes file from {fd}
There were 68820 Routes with IATA code
The EdgeList was succesfully created in variable edgeList


In [95]:
str(edgeList[0]).split(' ')

['edge:', 'AER', 'ASF', '1']

## Crazy Data Structure...

In [114]:
graph_dict = dict()
for edge in edgeList:
    temp = str(edge).split(' ')
    if temp[1] not in graph_dict:
        graph_dict[temp[1]] = [int(temp[3]), []]
    else:
        graph_dict[temp[1]][0] += int(temp[3])

for edge in edgeList:
    temp = str(edge).split(' ')
    if temp[2] in graph_dict:
        graph_dict[temp[2]][1].append((temp[1], temp[3]))
    else:
        pass

In [131]:
for i in graph_dict['AER'][1]:
    print(i[0])

ASF
GOJ
GYD
KRR
KZN
MCX
MRV
NBC
ROV
UUA
MMK
RTW
CSY
DME
SKX
ULV
KIV
MSQ
CEE
DYU
EVN
IST
LED
OVB
SVX
TAS
IJK
KVX
KBP
ALA
BAX
KEJ
KUF
NOZ
PEE
REN
TOF
VKO
SVO
KHV
KJA
NSK
TJM
EGO
VOZ


In [132]:
def computePageRanks(graph_dict, d=0.85, threshold=1e-8):
    pageRanks = dict()
    # initialize all 1/n vector
    for airport in graph_dict:
        pageRanks[airport] = 1/len(graph_dict)

    iterations = 0
    while (iterations < 100):
        for airport in graph_dict:
            pageRanks[airport] = sum([(pageRanks[i[0]] * i[1]) / (graph_dict[i[0]][0]) for i in graph_dict[airport][1]])

        # stopping condition
        rank_sum = sum([pageRanks[airport] for airport in pageRanks])
        if (abs(rank_sum - 1) > 1e-6):
            raise Exception(f'Rank sum != 1. Computed values is {rank_sum}')
        iterations += 1 

    return iterations, pageRanks


In [134]:
iterations, pageRanks = computePageRanks(graph_dict)

TypeError: can't multiply sequence by non-int of type 'float'

In [93]:
test_dict = {'JFK': [3, [('JFK', 7), ('EWR', 13), ('SCE', 21)]]}

if 'JFK' not in test_dict:
    print('doesnt work')
else:
    test_dict['JFK'][0] += 5
    test_dict['JFK'][1].append(('BCN', 4))
    print(test_dict['JFK'])

[8, [('JFK', 7), ('EWR', 13), ('SCE', 21), ('BCN', 4)]]


In [75]:
edgeList

[edge: AER ASF 1,
 edge: AER GOJ 1,
 edge: AER KRR 2,
 edge: AER KZN 2,
 edge: AER MCX 1,
 edge: AER MRV 1,
 edge: AER NBC 1,
 edge: AER ROV 1,
 edge: AER UUA 1,
 edge: ASF AER 1,
 edge: ASF KZN 1,
 edge: ASF LED 1,
 edge: CEK KZN 1,
 edge: CEK OVB 1,
 edge: DME MCX 1,
 edge: DME NBC 1,
 edge: DME TGK 1,
 edge: DME UUA 1,
 edge: GDZ KZN 1,
 edge: GOJ AER 1,
 edge: GYD AER 1,
 edge: GYD NBC 1,
 edge: KBP KZN 1,
 edge: KRR AER 2,
 edge: KRR KZN 1,
 edge: KUF MRV 1,
 edge: KZN AER 2,
 edge: KZN ASF 1,
 edge: KZN CEK 1,
 edge: KZN GDZ 1,
 edge: KZN KBP 1,
 edge: KZN KRR 1,
 edge: KZN LED 3,
 edge: KZN MRV 1,
 edge: KZN MUC 1,
 edge: KZN PEE 1,
 edge: KZN SIP 1,
 edge: LED ASF 1,
 edge: LED KZN 3,
 edge: LED MCX 2,
 edge: LED NBC 1,
 edge: LED UUA 1,
 edge: MCX AER 1,
 edge: MCX DME 1,
 edge: MCX LED 1,
 edge: MCX MRV 1,
 edge: MRV AER 1,
 edge: MRV KUF 1,
 edge: MRV KZN 1,
 edge: MRV MCX 1,
 edge: MUC KZN 1,
 edge: NBC AER 1,
 edge: NBC DME 1,
 edge: NBC GYD 1,
 edge: NBC LED 1,
 edge: NBC

In [56]:
for k, v in airportHash.items():
    print(k, v)

GKA 0
MAG 1
HGU 2
LAE 3
POM 4
WWK 5
UAK 6
GOH 7
SFJ 8
THU 9
AEY 10
EGS 11
HFN 12
HZK 13
IFJ 14
KEF 15
PFJ 16
RKV 17
SIJ 18
VEY 19
YAM 20
YAV 21
YAW 22
YAY 23
YAZ 24
YBB 25
YBC 26
YBG 27
YBK 28
YBL 29
YBR 30
YCB 31
YCD 32
YCG 33
YCH 34
YCL 35
YCO 36
YCT 37
YCW 38
YCY 39
YZS 40
YDA 41
YDB 42
YDC 43
YDF 44
YDL 45
YDN 46
YDQ 47
YEG 48
YEK 49
YEN 50
YET 51
YEU 52
YEV 53
YFB 54
YFC 55
YFO 56
YFR 57
YFS 58
YGK 59
YGL 60
YGP 61
YGQ 62
YGR 63
YHB 64
YHD 65
YHI 66
YHK 67
YHM 68
YHU 69
YHY 70
YHZ 71
YIB 72
YIO 73
YJN 74
YJT 75
YKA 76
YKF 77
YKL 78
YKY 79
YKZ 80
YLD 81
YLJ 82
YLL 83
YLT 84
YLW 85
YMA 86
YMJ 87
YMM 88
YMO 89
YMW 90
YMX 91
YNA 92
YND 93
YNM 94
YOC 95
YOD 96
YOJ 97
YOW 98
YPA 99
YPE 100
YPG 101
YPL 102
YPN 103
YPQ 104
YPR 105
YPY 106
YQA 107
YQB 108
YQF 109
YQG 110
YQH 111
YQK 112
YQL 113
YQM 114
YQQ 115
YQR 116
YQT 117
YQU 118
YQV 119
YQW 120
YQX 121
YQY 122
YQZ 123
YRB 124
YRI 125
YRJ 126
YRM 127
YRT 128
YSB 129
YSC 130
YSJ 131
YSM 132
YSR 133
YSU 134
YSY 135
YTE 136
YTH 137
YTR 13

In [61]:
x = [('JFK', 7), ('EWR', 13), ('SCE', 21)]

In [67]:
x[2][0]

'SCE'